In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.grid_search import RandomizedSearchCV
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import PCA
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression

/Users/kylemix/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/kylemix/anaconda/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [ ]:
pd.set_options

In [2]:
df = pd.read_csv('modelMe.csv')

In [3]:
df.head(1)

,Unnamed: 0,ActualBudget,ActualRevenue,genre1,runtime,DirMeanROI,meanGenreROI,genre3MeanROI,actorsMeanROI,year,month,dirExp,producer,actor1exp,actor2exp,actor3exp,actorsExp,crewsExp,ActualROI
0,0,41250000,355101932,16,87,7.608532,3.645812,4.807178,7.608532,2007,7,2,3,1,1,1,3,5,7.608532


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1444 entries, 0 to 1443
Data columns (total 18 columns):
ActualBudget     1444 non-null float64
ActualRevenue    1444 non-null float64
genre1           1444 non-null float64
runtime          1444 non-null float64
DirMeanROI       1444 non-null float64
meanGenreROI     1444 non-null float64
genre3MeanROI    1444 non-null float64
actorsMeanROI    1444 non-null float64
year             1444 non-null int64
month            1444 non-null int64
dirExp           1444 non-null float64
producer         1444 non-null float64
actor1exp        1444 non-null float64
actor2exp        1444 non-null float64
actor3exp        1444 non-null float64
actorsExp        1444 non-null float64
crewsExp         1444 non-null float64
ActualROI        1444 non-null float64
dtypes: float64(16), int64(2)
memory usage: 214.3 KB


In [4]:
df = df.drop(['Unnamed: 0'], axis = 1)


In [5]:

df = df.fillna(value=0)

In [6]:
df['winter'] = df['month'].map(lambda x: x in [12,1,2]).astype('int')
df['spring'] = df['month'].map(lambda x: x in [3,4,5]).astype('int')
df['summer'] = df['month'].map(lambda x: x in [6,7,8]).astype('int')
df['fall'] = df['month'].map(lambda x: x in [9,10,11]).astype('int')

In [7]:

set1=[1998,2006,2007,2008]
set2= [2009,2010,2011,2012,2013]
set3 = [2014,2015,2016]

In [8]:
df['earlyYear'] = df['year'].map(lambda x: x in set1).astype('int')
df['midYear'] = df['year'].map(lambda x: x in set2).astype('int')
df['lateYear'] = df['year'].map(lambda x: x in set3).astype('int')

4.4094196877005363

In [149]:
df.head()

,ActualBudget,ActualRevenue,genre1,runtime,DirMeanROI,meanGenreROI,genre3MeanROI,actorsMeanROI,year,month,...,actor3exp,actorsExp,crewsExp,winter,spring,summer,fall,earlyYear,midYear,lateYear
0,41250000,355101932,16,87,7.608532,3.645812,4.807178,7.608532,2007,7,...,1,3,5,0,0,1,0,1,0,0
1,101750000,767633591,18,152,4.979807,2.218555,2.070272,5.312557,2008,7,...,3,26,34,0,0,1,0,1,0,0
2,101750000,551868576,12,122,3.274444,1.919598,0.000000,2.651472,2008,5,...,15,37,43,0,1,0,0,1,0,0
3,165000000,635210212,12,169,1.327290,2.134808,1.841681,1.106099,2007,5,...,15,36,39,0,1,0,0,1,0,0
4,85000000,214233674,80,122,7.172342,3.390473,0.000000,3.451829,2007,6,...,16,42,43,0,0,1,0,1,0,0


In [9]:
df['dirRoiHigh'] = df.DirMeanROI.map(lambda x: x>=df.DirMeanROI.quantile(.8)).astype('int')
df['dirRoiMed'] = df.DirMeanROI.map(lambda x: df.DirMeanROI.quantile(.3)>=x>=df.DirMeanROI.quantile(.8)).astype('int')
df['dirRoiLow'] = df.DirMeanROI.map(lambda x: x<df.DirMeanROI.quantile(.3)).astype('int')

In [15]:
df['actorsMeanRoiHigh'] = df.actorsMeanROI.map(lambda x: x>=df.actorsMeanROI.quantile(.8)).astype('int')
df['actorsMeanRoiMid'] = df.actorsMeanROI.map(lambda x: df.actorsMeanROI.quantile(.3)>=x>=df.actorsMeanROI.quantile(.8)).astype('int')
df['actorsMeanRoiMid'] = df.actorsMeanROI.map(lambda x: x>=df.actorsMeanROI.quantile(.8)).astype('int')

In [16]:
# http://stackoverflow.com/questions/19377969/combine-two-columns-of-text-in-dataframe-in-pandas-python
# df['yearMonth'] = df.year.astype(str).str.cat(df.month.astype(str), sep=', ').astype('category')
# df['genre1'] = df.genre1.astype('category')
df.head(1)

,ActualBudget,ActualRevenue,genre1,runtime,DirMeanROI,meanGenreROI,genre3MeanROI,actorsMeanROI,year,month,...,summer,fall,earlyYear,midYear,lateYear,dirRoiHigh,dirRoiMed,dirRoiLow,actorsMeanRoiHigh,actorsMeanRoiMid
0,41250000,355101932,16,87,7.608532,3.645812,4.807178,7.608532,2007,7,...,1,0,1,0,0,1,0,0,1,1


In [17]:
genreDummies = pd.get_dummies(df.genre1)


In [18]:
genreDummies = genreDummies.drop(18,axis=1)

In [19]:
genreDummies.head(1)

,0.0,12.0,14.0,16.0,27.0,28.0,35.0,36.0,37.0,53.0,80.0,99.0,878.0,9648.0,10402.0,10749.0,10751.0,10752.0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
target = df.ActualROI
df= df.drop(['ActualROI'], axis = 1)

AttributeError: 'DataFrame' object has no attribute 'ActualROI'

In [21]:
df1 = pd.concat([df,genreDummies], axis=1)
df1 = df1.drop(['genre1', 'year','month','actorsMeanROI'],axis=1)

In [37]:
Xval = df1
Xval2 = df1.drop('DirMeanROI', axis=1)
Xval3 = df1.drop(['ActualRevenue','DirMeanROI'], axis=1)

In [43]:
Xval3.columns

Index([     'ActualBudget',           'runtime',      'meanGenreROI',
           'genre3MeanROI',            'dirExp',          'producer',
               'actor1exp',         'actor2exp',         'actor3exp',
               'actorsExp',          'crewsExp',            'winter',
                  'spring',            'summer',              'fall',
               'earlyYear',           'midYear',          'lateYear',
              'dirRoiHigh',         'dirRoiMed',         'dirRoiLow',
       'actorsMeanRoiHigh',  'actorsMeanRoiMid',                 0.0,
                      12.0,                14.0,                16.0,
                      27.0,                28.0,                35.0,
                      36.0,                37.0,                53.0,
                      80.0,                99.0,               878.0,
                    9648.0,             10402.0,             10749.0,
                   10751.0,             10752.0],
      dtype='object')

In [54]:
# from x removed:,'DirMeanROI','runtime','actorsMeanROI','genre3MeanROI','DirMeanROI','actor1exp','actor2exp','actor3exp',
# t1 = df[['ActualBudget','ActualRevenue','dirExp','meanGenreROI',
#         'dirExp','producer','actorsExp','crewsExp','DirMeanROI']]
# t2 = df[['ActualBudget','ActualRevenue','dirExp','meanGenreROI',
#         'dirExp','producer','actorsExp','crewsExp']]

X = Xval3
y = target
cols = Xval3.columns

In [173]:
lr1 = LinearRegression(n_jobs=True)

In [174]:
scores = cross_val_score(lr1, X, y,cv=10,scoring = 'r2',n_jobs = -1)
print('scores= ',scores, 'mean=',scores.mean())

scores=  [-0.53644459  0.16359618  0.30743996  0.33722029  0.32659812  0.20870229
  0.38243679 -0.01066331  0.40213114  0.43381854] mean= 0.201483541154


In [140]:
linfit = lr1.fit(X,y)

In [141]:
linfit.coef_

array([ -1.78628004e-08,  -1.43215416e-02,   2.85080941e+00,
        -5.68467113e-02,   1.39549930e+00,   1.33855340e-01,
        -5.40676259e-01,  -5.72554430e-01,  -5.54703145e-01,
         1.75673250e+00,  -1.26102670e+00,  -1.77525645e-01,
        -4.81058800e-01,   3.46777016e-01,   3.11807426e-01,
        -4.07217189e-01,   1.51348564e-01,   2.55868623e-01,
         1.08357844e+01,   1.75896293e+00,   1.07220948e+00,
         4.89550467e-01,  -4.05332254e+00,   7.98498551e-01,
         1.14824495e-01,   9.11450010e-01,   4.74212548e-01,
        -1.03519014e-01,   7.96675836e-01,  -2.33028203e+01,
        -1.81167878e-01,   6.22365693e-01,   7.30129529e-01,
         4.35826290e-01,  -1.30624009e-02,   1.36684975e+00])

In [196]:
# rand = RandomizedSearchCV(lr1, cv=10, scoring='r2',n_iter = 10,random_state=42)
# rand.fit(linX,liny)
# rand.grid_scores_

TypeError: __init__() missing 1 required positional argument: 'param_distributions'

In [51]:

X.columns

Index([     'ActualBudget',           'runtime',      'meanGenreROI',
           'genre3MeanROI',            'dirExp',          'producer',
               'actor1exp',         'actor2exp',         'actor3exp',
               'actorsExp',          'crewsExp',            'winter',
                  'spring',            'summer',              'fall',
               'earlyYear',           'midYear',          'lateYear',
              'dirRoiHigh',         'dirRoiMed',         'dirRoiLow',
       'actorsMeanRoiHigh',  'actorsMeanRoiMid',                 0.0,
                      12.0,                14.0,                16.0,
                      27.0,                28.0,                35.0,
                      36.0,                37.0,                53.0,
                      80.0,                99.0,               878.0,
                    9648.0,             10402.0,             10749.0,
                   10751.0,             10752.0],
      dtype='object')

In [48]:
randF300 = RandomForestRegressor(n_estimators=300, random_state=42)

In [45]:
rf.fit(Xval3,target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=300, n_jobs=1, oob_score=False, random_state=42,
           verbose=0, warm_start=False)

In [55]:
randF300.fit(X,y)
randF300.feature_importances_
sorted(zip(rf.feature_importances_, cols),reverse=True)

[(0.22344680341140469, 'ActualBudget'),
 (0.16110540433857076, 'actorsMeanRoiHigh'),
 (0.1597592390632494, 'actorsMeanRoiMid'),
 (0.15172499927373689, 'meanGenreROI'),
 (0.044219832349371081, 'runtime'),
 (0.029408785787397296, 'dirRoiHigh'),
 (0.024737906456757382, 'producer'),
 (0.02146967627975728, 'dirExp'),
 (0.0210448333330162, 'actor1exp'),
 (0.020888146632341155, 'actorsExp'),
 (0.019206766254908903, 'crewsExp'),
 (0.016347796488853573, 'dirRoiLow'),
 (0.014062638972067616, 99.0),
 (0.013987107227976265, 'actor2exp'),
 (0.012920359239964345, 'genre3MeanROI'),
 (0.010178032891768092, 'actor3exp'),
 (0.0090849686792346483, 'fall'),
 (0.007545322486254071, 'earlyYear'),
 (0.0050118114163378363, 'spring'),
 (0.0045714430424508784, 'midYear'),
 (0.0041384350717403318, 'winter'),
 (0.0031975890980258923, 35.0),
 (0.0031194742920568931, 14.0),
 (0.0027976193062729717, 53.0),
 (0.0027398956395651981, 'lateYear'),
 (0.0025666537576187723, 'summer'),
 (0.0024586271048825312, 80.0),
 (0.0

In [49]:
scores = cross_val_score(randF300, X, y,cv=10,scoring = 'r2',n_jobs = -1)
print('scores= ',scores, 'mean=',scores.mean())

scores=  [ 0.67768868  0.34087998  0.60725009  0.61685622  0.69519157  0.5995015
  0.60656168  0.26841481  0.62864307  0.36153697] mean= 0.54025245784


In [93]:
rf2 = RandomForestRegressor(n_estimators=200)
rf2.fit(Xval2,y)
rf2.feature_importances_
sorted(zip(rf2.feature_importances_, cols),reverse=True)

[(0.43284264525024801, 'ActualRevenue'),
 (0.4056940568583437, 'ActualBudget'),
 (0.044398005276497142, 'DirMeanROI'),
 (0.025341570971592238, 'runtime'),
 (0.015579997249064481, 14.0),
 (0.013527285460164901, 10752.0),
 (0.0091276439657972091, 'actorsExp'),
 (0.007662736555970725, 'genre3MeanROI'),
 (0.0073821081601994328, 'actor3exp'),
 (0.005535706161935933, 'dirExp'),
 (0.0054400991510449382, 'actor2exp'),
 (0.0041257823455429828, 'producer'),
 (0.0038689463473881702, 'actor1exp'),
 (0.0036009459959806152, 0.0),
 (0.0033359103257834472, 'meanGenreROI'),
 (0.0026938249406605395, 16.0),
 (0.0018595726461929931, 27.0),
 (0.0011531866227655655, 28.0),
 (0.0011213534186060275, 10751.0),
 (0.0011112498334896531, 12.0),
 (0.0010173284048436444, 'crewsExp'),
 (0.00097108618394070026, 99.0),
 (0.00063693316560165132, 878.0),
 (0.00062185768343824086, 80.0),
 (0.00057690298320247886, 10749.0),
 (0.00033205339395665339, 37.0),
 (0.00030523619423007334, 35.0),
 (3.1339185987484527e-05, '2007, 

In [98]:
Xval2.columns

Index([ 'ActualBudget', 'ActualRevenue',       'runtime',  'meanGenreROI',
       'genre3MeanROI',        'dirExp',      'producer',     'actor1exp',
           'actor2exp',     'actor3exp',     'actorsExp',      'crewsExp',
              'winter',        'spring',        'summer',          'fall',
           'earlyYear',       'midYear',      'lateYear',             0.0,
                  12.0,            14.0,            16.0,            27.0,
                  28.0,            35.0,            36.0,            37.0,
                  53.0,            80.0,            99.0,           878.0,
                9648.0,         10402.0,         10749.0,         10751.0,
               10752.0],
      dtype='object')

In [28]:
lass = Lasso(alpha = .5)

In [29]:
scores = cross_val_score(lass, X, y,cv=10,scoring = 'r2',n_jobs = -1)
print('scores= ',scores, 'mean=',scores.mean())

scores=  [ 0.1615236   0.25411388  0.35671963  0.39862209  0.31318359  0.27362806
  0.35791866  0.19922462  0.44261008  0.52518577] mean= 0.328272998396
